# Working with Iris data set

In [1]:
#Ignore warning msgs
import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")

### SECTION 1 : Load and setup data for training and testing

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [3]:
data = pd.read_csv('iris_shuffle.csv')

#Split train & test of 20%
iris_train, iris_test = train_test_split(data, test_size=0.2)

#Storing to individual csv files
iris_train.to_csv('iris_train.csv', index = False)
iris_test.to_csv('iris_test.csv', index = False)

#### Preparing train data set

In [4]:
datatrain = pd.read_csv('iris_train.csv')
#change string value to numeric
species_numeric = {'setosa': 0,'versicolor':1,'virginica':2}
datatrain.species = [species_numeric[item] for item in datatrain.species] 
#datatrain.head()
#change dataframe to array
datatrain_array = datatrain.as_matrix()

#split x and y (feature and target)
xtrain = datatrain_array[:,:-1]
ytrain = datatrain_array[:,-1].astype(int)

#print data summary
#len(xtrain), len(ytrain)

#### Preparing test data set

In [5]:
#load data test
datatest = pd.read_csv('iris_test.csv')

#change string value to numeric
species_numeric = {'setosa': 0,'versicolor':1,'virginica':2}
datatest.species = [species_numeric[item] for item in datatest.species] 
datatest.head()

#change dataframe to array
datatest_array = datatest.as_matrix()

#split x and y (feature and target)
xtest = datatest_array[:,:-1]
ytest = datatest_array[:,-1].astype(int)

#print data
#len(xtest), len(ytest)
#xtrain.shape, ytrain.shape, xtest.shape, ytest.shape

### SECTION 2 :Tensorflow configuration

In [6]:
import os
import tensorflow as tf

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # minimzing logging
cwd = os.path.abspath(os.path.dirname('iris')) # path for saving model
saver_path = os.path.abspath(os.path.join(cwd, 'models/model_sess.ckpt'))
tf.set_random_seed(1103) # avoiding different result of random

# tensorflow model
input_data = tf.placeholder(tf.float32, [None, 4], name="input_data")
label = tf.placeholder(tf.float32, [None], name ="label")
onehot_label = tf.one_hot(tf.cast(label, tf.int32), 3, name="onehot_label")
hidden = tf.layers.dense(input_data, 16, tf.nn.relu, name="hidden")
output = tf.layers.dense(hidden, 3, tf.nn.relu, name="output")
soft_output = tf.nn.softmax(output, name="soft_output")

init = tf.global_variables_initializer()
saver = tf.train.Saver()

# optimization
loss = -tf.reduce_sum(onehot_label * tf.log(soft_output))
optimizer = tf.train.GradientDescentOptimizer(0.0001)
is_correct = tf.equal(tf.argmax(soft_output,1), tf.argmax(onehot_label,1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
train_step = optimizer.minimize(loss)

#### Train the model 

In [7]:
with tf.Session() as sess:
    sess.run(init)
    
    # start training
    for e in range(1000):
        if(e%100==0):
            print(e,"/ 1000 - Loss:", sess.run(loss, feed_dict={input_data:xtrain, label:ytrain}))
        sess.run(train_step, feed_dict={input_data:xtrain, label:ytrain})
    
    # save model
    saver.save(sess, saver_path)
    print("Train accuracy",sess.run(accuracy, feed_dict={input_data:xtrain, label:ytrain}))

0 / 1000 - Loss: 133.96713
100 / 1000 - Loss: 67.151634
200 / 1000 - Loss: 53.1354
300 / 1000 - Loss: 44.28194
400 / 1000 - Loss: 38.113697
500 / 1000 - Loss: 33.108505
600 / 1000 - Loss: 28.993217
700 / 1000 - Loss: 25.646944
800 / 1000 - Loss: 22.946606
900 / 1000 - Loss: 20.769405
Train accuracy 0.98333335


#### Test the model

In [8]:
with tf.Session() as sess:
    saver.restore(sess, saver_path)
    sess.run(accuracy, feed_dict={input_data:xtest, label:ytest})
    print("Test accuracy",sess.run(accuracy, feed_dict={input_data:xtest, label:ytest}))

INFO:tensorflow:Restoring parameters from /home/reva/Documents/machine_leaning_cs231/iris/models/model_sess.ckpt
Test accuracy 0.96666664


#### Print all available tensors in graph

In [17]:
from tensorflow.python.tools.inspect_checkpoint import print_tensors_in_checkpoint_file

with tf.Session() as sess:
    saver = tf.train.import_meta_graph("models/model_sess.ckpt.meta")
    saver.restore(sess, saver_path)
    print_tensors_in_checkpoint_file(file_name="models/model_sess.ckpt", tensor_name='', all_tensors=False)

INFO:tensorflow:Restoring parameters from /home/reva/Documents/machine_leaning_cs231/iris/models/model_sess.ckpt
hidden/bias (DT_FLOAT) [16]
hidden/kernel (DT_FLOAT) [4,16]
output/bias (DT_FLOAT) [3]
output/kernel (DT_FLOAT) [16,3]




#### Print all operations in graph

In [20]:
    graph=tf.get_default_graph()
    print ("")
    for op in graph.get_operations():  
        print (str(op.name))


input_data
label
Cast
onehot_label/on_value
onehot_label/off_value
onehot_label/depth
onehot_label
hidden/kernel/Initializer/random_uniform/shape
hidden/kernel/Initializer/random_uniform/min
hidden/kernel/Initializer/random_uniform/max
hidden/kernel/Initializer/random_uniform/RandomUniform
hidden/kernel/Initializer/random_uniform/sub
hidden/kernel/Initializer/random_uniform/mul
hidden/kernel/Initializer/random_uniform
hidden/kernel
hidden/kernel/Assign
hidden/kernel/read
hidden/bias/Initializer/zeros
hidden/bias
hidden/bias/Assign
hidden/bias/read
hidden/MatMul
hidden/BiasAdd
hidden/Relu
output/kernel/Initializer/random_uniform/shape
output/kernel/Initializer/random_uniform/min
output/kernel/Initializer/random_uniform/max
output/kernel/Initializer/random_uniform/RandomUniform
output/kernel/Initializer/random_uniform/sub
output/kernel/Initializer/random_uniform/mul
output/kernel/Initializer/random_uniform
output/kernel
output/kernel/Assign
output/kernel/read
output/bias/Initializer/zer